In [113]:
#importing necessary libraries
import pandas as pd
import numpy as np
from functools import reduce
from pathlib import Path

In [114]:
#defining directories
dir_path=Path.cwd().parents[1]
masterlist_path=dir_path.joinpath('data_docs/golden_states_masterlist')
appended_masterlist=dir_path.joinpath('data_docs/appended_masterlist.xlsx')
masterlist_log_file=dir_path.joinpath('data_docs/masterlist_log_file.xlsx')

In [115]:
def master_reader():

    files=list(masterlist_path.glob("*"))[1:]
    print(files)
    dfs=[pd.read_excel(file, sheet_name=x, engine='openpyxl', ) for file in files for x in [0,1,2]]
    
    df=pd.concat(dfs, axis=0)

    # df.to_excel(appended_masterlist, index=False)

    return df



In [116]:
df=master_reader()
df.columns


[WindowsPath('d:/work_isb/projects/Voting/data_docs/golden_states_masterlist/andhra_pradesh_masterlist.xlsx'), WindowsPath('d:/work_isb/projects/Voting/data_docs/golden_states_masterlist/assam_masterlist.xlsx'), WindowsPath('d:/work_isb/projects/Voting/data_docs/golden_states_masterlist/chhattisgarh_masterlist.xlsx'), WindowsPath('d:/work_isb/projects/Voting/data_docs/golden_states_masterlist/goa_masterlist.xlsx'), WindowsPath('d:/work_isb/projects/Voting/data_docs/golden_states_masterlist/gujarat_masterlist.xlsx'), WindowsPath('d:/work_isb/projects/Voting/data_docs/golden_states_masterlist/himachal_pradesh_masterlist.xlsx'), WindowsPath('d:/work_isb/projects/Voting/data_docs/golden_states_masterlist/jharkhand_masterlist.xlsx'), WindowsPath('d:/work_isb/projects/Voting/data_docs/golden_states_masterlist/karnataka_masterlist.xlsx'), WindowsPath('d:/work_isb/projects/Voting/data_docs/golden_states_masterlist/kerala_masterlist.xlsx'), WindowsPath('d:/work_isb/projects/Voting/data_docs/gol

d:\work_isb\projects\Voting\.venv\lib\site-packages\openpyxl\worksheet\_reader.py:335: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
d:\work_isb\projects\Voting\.venv\lib\site-packages\openpyxl\worksheet\_reader.py:335: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
d:\work_isb\projects\Voting\.venv\lib\site-packages\openpyxl\worksheet\_reader.py:335: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
d:\work_isb\projects\Voting\.venv\lib\site-packages\openpyxl\worksheet\_reader.py:335: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
d:\work_isb\projects\Voting\.venv\lib\site-packages\openpyxl\worksheet\_reader.py:335: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
d:\work_isb\projects\Voting\.venv\lib\site-packages\openpyxl\worksheet\_reader.py:335: UserWarning: Data Validation extension

Index(['year', 'election', 'ac_no', 'state', 'ac_name', 'downloaded',
       'converted', 'cleaned', 'image', 'indic', 'file_status',
       'error_comment', 'bye_election', 'polling_station_name', 'note'],
      dtype='object')

In [117]:
total_acs=df.groupby(
    [
        'state',
        'bye_election',
        'year',
    ]
)['ac_name'].agg('count').reset_index()

total_acs.rename(columns={'ac_name':'number_of_ac'}, inplace=True)
total_acs

,state,bye_election,year,number_of_ac
0,Andhra Pradesh,0,2014,175
1,Andhra Pradesh,0,2019,175
2,Andhra Pradesh,1,2015,1
3,Andhra Pradesh,1,2017,1
4,Andhra Pradesh,1,2021,1
...,...,...,...,...
104,kerala,1,2018,1
105,kerala,1,2019,6
106,kerala,1,2021,2
107,kerala,1,2022,1


In [118]:
targets=['downloaded','converted','cleaned','image','indic','polling_station_name']

data_list=[]
data_list.append(total_acs)
index_vars=[
        'state',
        'bye_election',
        'year'
        ]


for var in targets:
    
    #grouping the entire data frame by state, by_election dummy and year.
    grouped_df=df.groupby(index_vars)[var].value_counts()

    # widneing the vakue counts to generate percentagfe of each columns in targets
    widened_df=pd.pivot(grouped_df.reset_index(),index=index_vars, columns=var).reset_index()
    col_vars=index_vars + ['0', '1']
    widened_df.columns=col_vars

    #replacing missing values in 0 and 1 with 0. they are missing becuase some states have all images or none downloaded etc.
    widened_df['0']=np.where(widened_df['0'].isna(), 0, widened_df['0'])
    widened_df['1']=np.where(widened_df['1'].isna(), 0, widened_df['1'])  

    #creating total and percent (progress) in the widended data
    widened_df['total']=widened_df['0']+widened_df['1'] 
    widened_df['percent']=round((widened_df['1']/widened_df['total'])*100, 0)

    # cleaning the column names
    col_vars=widened_df.columns
    col_vars=[x.replace('1', var) if x=='1'else x for x in col_vars]
    col_vars=['_'.join([var, x]) if x=='percent' else x for x in col_vars]

    # removing the 0 column and total
    widened_df.columns=col_vars
    widened_df.drop(['0', 'total'], axis=1, inplace=True)
    data_list.append(widened_df)



merged_df=reduce(lambda x, y: pd.merge(x, y, how='outer', on=index_vars, validate='1:1'), data_list)




In [119]:
merged_df.to_excel(masterlist_log_file, index=False)